In [44]:
from dash import Dash, html, dcc, callback, Output, Input
from dash_bootstrap_templates import load_figure_template
import dash_bootstrap_components as dbc
from dash import dash_table
import plotly_express as px
import pandas as pd

In [6]:
stocks = pd.read_csv("../Data/stocks.csv", parse_dates=["Date"], index_col="Date")
stocks.head(2)

,Symbols,Close,High,Low,Open,Volume
Date,,,,,,
2023-01-03,AAPL,125.07,130.90,124.17,130.28,112117471
2023-01-03,MSFT,239.58,245.75,237.40,243.08,25740036


In [52]:
load_figure_template("flatly")

app = Dash(
    __name__, 
    external_stylesheets=[dbc.themes.FLATLY],
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Stock Market Dashboard", className='text-center text-primary mx-3')
        ], width=12)
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id="single_dropdown", multi=False, searchable=False, className='mb-2', value='MSFT',
                options=[stock for stock in stocks["Symbols"].unique()],
                style={'color': '#333'}
            ),
            dcc.Graph(id="volume_graph", config={"staticPlot": True},
                figure={})
        ], xs=12, sm=11, md=10, lg=5),

        dbc.Col([
            dcc.Dropdown(
                id="multi_dropdown", multi=True, searchable=False, className='mb-2',
                options=[stock for stock in stocks["Symbols"].unique()],
                style={'color': '#333'}
            ),
            dcc.Graph(id="closing_graph", config={"staticPlot": True},
                figure={})
        ], xs=12, sm=11, md=10, lg=5)
    ], justify="evenly"),

    dbc.Row([
        dbc.Col([
            dash_table.DataTable(stocks.query("Symbols=='MSFT'").to_dict('records'), page_size=15)
        ], width=10)
    ], justify='center'),


], fluid=True)

@callback(
    Output("volume_graph", "figure"),
    Input("single_dropdown", "value")
)
def update_volume_graph(symbol):
    df = stocks.query("Symbols==@symbol")
    return px.line(df, x=df.index, y="Volume")

@callback(
    Output("closing_graph", "figure"),
    Input("multi_dropdown", "value")
)
def update_closing_graph(symbols):
    if symbols is None: symbols = []
    df = stocks[stocks["Symbols"].isin(symbols)]
    return px.line(df, x=df.index, y="Close", color="Symbols")


app.run(debug=True, jupyter_mode='external')


Dash app running on http://127.0.0.1:8050/
